# TinyYolov4 for Syringe Dataset

#### Mount Drive

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


#### Check Drive Version

In [ ]:
!nvcc --version
!nvidia-smi

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2020 NVIDIA Corporation
Built on Wed_Jul_22_19:09:09_PDT_2020
Cuda compilation tools, release 11.0, V11.0.221
Build cuda_11.0_bu.TC445_37.28845127_0
Sat Aug 28 09:39:27 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.57.02    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   34C    P8     9W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |         

#### Unzip Darknet Zip File

In [2]:
# Unzip darknet file and images
!unzip "/content/drive/MyDrive/MLData/tiny-yolov4-data/tiny-yolov4-darknet.zip"

Archive:  /content/drive/MyDrive/MLData/tiny-yolov4-data/tiny-yolov4-darknet.zip
   creating: darknet/
   creating: darknet/.circleci/
  inflating: darknet/.circleci/config.yml  
   creating: darknet/.git/
   creating: darknet/.github/
  inflating: darknet/.github/FUNDING.yml  
   creating: darknet/.github/ISSUE_TEMPLATE/
  inflating: darknet/.github/ISSUE_TEMPLATE/any-other-question-or-issue.md  
  inflating: darknet/.github/ISSUE_TEMPLATE/bug-report.md  
  inflating: darknet/.github/ISSUE_TEMPLATE/feature_request.md  
  inflating: darknet/.github/ISSUE_TEMPLATE/training-issue---no-detections---nan-avg-loss---low-accuracy.md  
   creating: darknet/.github/workflows/
  inflating: darknet/.github/workflows/ccpp.yml  
  inflating: darknet/.github/workflows/on_pr.yml  
  inflating: darknet/.github/workflows/rebase.yml  
  inflating: darknet/.gitignore      
  inflating: darknet/.git/config     
  inflating: darknet/.git/description  
 extracting: darknet/.git/HEAD       
   creating: dark

#### Change directory and download training images, unzip them and delete the zip file. Also download TinyYolov4 weights


In [3]:
%cd '/content/darknet'
!wget https://github.com/AlexeyAB/darknet/releases/download/darknet_yolo_v4_pre/yolov4-tiny.conv.29
!curl -L "https://app.roboflow.com/ds/qEQtFr1Abp?key=KCfPQpl5VT" > roboflow.zip; unzip roboflow.zip; rm roboflow.zip

Streaming output truncated to the last 5000 lines.
 extracting: train/IMG_1800_JPG_jpg.rf.3ea398948890f945de8b8fd4f968ad0b.txt  
 extracting: train/IMG_1800_JPG_jpg.rf.b044aac10a2d0d6141ddba21ebf4bf5a.jpg  
 extracting: train/IMG_1800_JPG_jpg.rf.b044aac10a2d0d6141ddba21ebf4bf5a.txt  
 extracting: train/IMG_1800_JPG_jpg.rf.e206ac23ba83112348954bad5acc5f32.jpg  
 extracting: train/IMG_1800_JPG_jpg.rf.e206ac23ba83112348954bad5acc5f32.txt  
 extracting: train/IMG_1800_JPG_jpg.rf.e41312ba6cecdc8994107496a219ba73.jpg  
 extracting: train/IMG_1800_JPG_jpg.rf.e41312ba6cecdc8994107496a219ba73.txt  
 extracting: train/IMG_1800_JPG_jpg.rf.f810d95fe3403139e238d837a19c4f6f.jpg  
 extracting: train/IMG_1800_JPG_jpg.rf.f810d95fe3403139e238d837a19c4f6f.txt  
 extracting: train/IMG_1801_JPG_jpg.rf.4bce676b2039b350ffa8ee6ef3fd4a95.jpg  
 extracting: train/IMG_1801_JPG_jpg.rf.4bce676b2039b350ffa8ee6ef3fd4a95.txt  
 extracting: train/IMG_1801_JPG_jpg.rf.4c6e0e09f56d7973dd9cf7f25113b6c6.jpg  
 extracting: 

#### Make install darknet and give access permission to the installer

In [4]:
!make
!chmod +x ./darknet

mkdir -p ./obj/
mkdir -p backup
chmod +x *.sh
g++ -std=c++11 -std=c++11 -Iinclude/ -I3rdparty/stb/include -DGPU -I/usr/local/cuda/include/ -Wall -Wfatal-errors -Wno-unused-result -Wno-unknown-pragmas -fPIC -Ofast -DGPU -c ./src/image_opencv.cpp -o obj/image_opencv.o
g++ -std=c++11 -std=c++11 -Iinclude/ -I3rdparty/stb/include -DGPU -I/usr/local/cuda/include/ -Wall -Wfatal-errors -Wno-unused-result -Wno-unknown-pragmas -fPIC -Ofast -DGPU -c ./src/http_stream.cpp -o obj/http_stream.o
./src/http_stream.cpp: In member function ‘bool JSON_sender::write(const char*)’:
./src/http_stream.cpp:253:21: warning: unused variable ‘n’ [-Wunused-variable]
                 int n = _write(client, outputbuf, outlen);
                     ^
./src/http_stream.cpp: In function ‘void set_track_id(detection*, int, float, float, float, int, int, int)’:
./src/http_stream.cpp:867:27: warning: comparison between signed and unsigned integer expressions [-Wsign-compare]
         for (int i = 0; i < v.size(); ++i) {


#### Prepare data file and text files for training

In [6]:
#Set up training file directories for custom dataset

%cp train/_darknet.labels data/obj.names
%mkdir data/obj
#copy image and labels
%cp train/*.jpg data/obj/
%cp valid/*.jpg data/obj/

%cp train/*.txt data/obj/
%cp valid/*.txt data/obj/

with open('data/obj.data', 'w') as out:
  out.write('classes = 2\n') ##CHANGE CLASS NUMBER HERE!!
  out.write('train = data/train.txt\n')
  out.write('valid = data/valid.txt\n')
  out.write('names = data/obj.names\n')
  out.write('backup = backup/')

#write train file (just the image list)
import os

with open('data/train.txt', 'w') as out:
  for img in [f for f in os.listdir('train') if f.endswith('jpg')]:
    out.write('data/obj/' + img + '\n')

#write the valid file (just the image list)
import os

with open('data/valid.txt', 'w') as out:
  for img in [f for f in os.listdir('valid') if f.endswith('jpg')]:
    out.write('data/obj/' + img + '\n')

#### Write custom CFG file

In [7]:
# Custom CFG file
#we build config dynamically based on number of classes
#we build iteratively from base config files. This is the same file shape as cfg/yolo-obj.cfg
def file_len(fname):
  with open(fname) as f:
    for i, l in enumerate(f):
      pass
  return i + 1

num_classes = file_len('train/_darknet.labels')
max_batches = 10000
steps1 = .8 * max_batches
steps2 = .9 * max_batches
steps_str = str(int(steps1))+','+str(int(steps2))
num_filters = (num_classes + 5) * 3


print("writing config for a custom YOLOv4 detector detecting number of classes: " + str(num_classes))

#Instructions from the darknet repo
#change line max_batches to (classes*2000 but not less than number of training images, and not less than 6000), f.e. max_batches=6000 if you train for 3 classes
#change line steps to 80% and 90% of max_batches, f.e. steps=4800,5400
if os.path.exists('./cfg/yolov4-tiny-custom.cfg'): os.remove('./cfg/yolov4-tiny-custom.cfg.cfg')


#customize iPython writefile so we can write variables
from IPython.core.magic import register_line_cell_magic

@register_line_cell_magic
def writetemplate(line, cell):
    with open(line, 'w') as f:
        f.write(cell.format(**globals()))

writing config for a custom YOLOv4 detector detecting number of classes: 2


#### Write CFG file

In [8]:
%%writetemplate ./cfg/yolov4-tiny-obj.cfg
[net]
# Testing
#batch=1
#subdivisions=1
# Training
batch=64
subdivisions=1
width=416
height=416
channels=3
momentum=0.9
decay=0.0005
angle=0
saturation=1.5
exposure=1.5
hue=.1

learning_rate=0.00261
burn_in=1000
max_batches={max_batches}
policy=steps
steps={steps_str}
scales=.1,.1

[convolutional]
batch_normalize=1
filters=32
size=3
stride=2
pad=1
activation=leaky

[convolutional]
batch_normalize=1
filters=64
size=3
stride=2
pad=1
activation=leaky

[convolutional]
batch_normalize=1
filters=64
size=3
stride=1
pad=1
activation=leaky

[route]
layers=-1
groups=2
group_id=1

[convolutional]
batch_normalize=1
filters=32
size=3
stride=1
pad=1
activation=leaky

[convolutional]
batch_normalize=1
filters=32
size=3
stride=1
pad=1
activation=leaky

[route]
layers = -1,-2

[convolutional]
batch_normalize=1
filters=64
size=1
stride=1
pad=1
activation=leaky

[route]
layers = -6,-1

[maxpool]
size=2
stride=2

[convolutional]
batch_normalize=1
filters=128
size=3
stride=1
pad=1
activation=leaky

[route]
layers=-1
groups=2
group_id=1

[convolutional]
batch_normalize=1
filters=64
size=3
stride=1
pad=1
activation=leaky

[convolutional]
batch_normalize=1
filters=64
size=3
stride=1
pad=1
activation=leaky

[route]
layers = -1,-2

[convolutional]
batch_normalize=1
filters=128
size=1
stride=1
pad=1
activation=leaky

[route]
layers = -6,-1

[maxpool]
size=2
stride=2

[convolutional]
batch_normalize=1
filters=256
size=3
stride=1
pad=1
activation=leaky

[route]
layers=-1
groups=2
group_id=1

[convolutional]
batch_normalize=1
filters=128
size=3
stride=1
pad=1
activation=leaky

[convolutional]
batch_normalize=1
filters=128
size=3
stride=1
pad=1
activation=leaky

[route]
layers = -1,-2

[convolutional]
batch_normalize=1
filters=256
size=1
stride=1
pad=1
activation=leaky

[route]
layers = -6,-1

[maxpool]
size=2
stride=2

[convolutional]
batch_normalize=1
filters=512
size=3
stride=1
pad=1
activation=leaky

##################################

[convolutional]
batch_normalize=1
filters=256
size=1
stride=1
pad=1
activation=leaky

[convolutional]
batch_normalize=1
filters=512
size=3
stride=1
pad=1
activation=leaky

[convolutional]
size=1
stride=1
pad=1
filters={num_filters}
activation=linear



[yolo]
mask = 3,4,5
anchors = 10,14,  23,27,  37,58,  81,82,  135,169,  344,319
classes={num_classes}
num=6
jitter=.3
scale_x_y = 1.05
cls_normalizer=1.0
iou_normalizer=0.07
iou_loss=ciou
ignore_thresh = .7
truth_thresh = 1
random=0
resize=1.5
nms_kind=greedynms
beta_nms=0.6

[route]
layers = -4

[convolutional]
batch_normalize=1
filters=128
size=1
stride=1
pad=1
activation=leaky

[upsample]
stride=2

[route]
layers = -1, 23

[convolutional]
batch_normalize=1
filters=256
size=3
stride=1
pad=1
activation=leaky

[convolutional]
size=1
stride=1
pad=1
filters={num_filters}
activation=linear

[yolo]
mask = 0,1,2
anchors = 10,14,  23,27,  37,58,  81,82,  135,169,  344,319
classes={num_classes}
num=6
jitter=.3
scale_x_y = 1.05
cls_normalizer=1.0
iou_normalizer=0.07
iou_loss=ciou
ignore_thresh = .7
truth_thresh = 1
random=0
resize=1.5
nms_kind=greedynms
beta_nms=0.6

#### Print out CFG files

In [9]:
%cat cfg/yolov4-tiny-obj.cfg

[net]
# Testing
#batch=1
#subdivisions=1
# Training
batch=64
subdivisions=1
width=416
height=416
channels=3
momentum=0.9
decay=0.0005
angle=0
saturation=1.5
exposure=1.5
hue=.1

learning_rate=0.00261
burn_in=1000
max_batches=10000
policy=steps
steps=8000,9000
scales=.1,.1

[convolutional]
batch_normalize=1
filters=32
size=3
stride=2
pad=1
activation=leaky

[convolutional]
batch_normalize=1
filters=64
size=3
stride=2
pad=1
activation=leaky

[convolutional]
batch_normalize=1
filters=64
size=3
stride=1
pad=1
activation=leaky

[route]
layers=-1
groups=2
group_id=1

[convolutional]
batch_normalize=1
filters=32
size=3
stride=1
pad=1
activation=leaky

[convolutional]
batch_normalize=1
filters=32
size=3
stride=1
pad=1
activation=leaky

[route]
layers = -1,-2

[convolutional]
batch_normalize=1
filters=64
size=1
stride=1
pad=1
activation=leaky

[route]
layers = -6,-1

[maxpool]
size=2
stride=2

[convolutional]
batch_normalize=1
filters=128
size=3
stride=1
pad=1
activation=leaky

[route]
layers=-

#### Convert all files to UNIX

In [10]:
!sudo apt install dos2unix
!dos2unix data/obj.data
!dos2unix cfg/yolov4-tiny-obj.cfg
!dos2unix data/obj.names
!dos2unix data/train.txt
!dos2unix data/valid.txt

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'sudo apt autoremove' to remove it.
The following NEW packages will be installed:
  dos2unix
0 upgraded, 1 newly installed, 0 to remove and 40 not upgraded.
Need to get 351 kB of archives.
After this operation, 1,267 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 dos2unix amd64 7.3.4-3 [351 kB]
Fetched 351 kB in 1s (370 kB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 76, <> line 1.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletype
dpkg-preconf

#### Train TinyYolov4 Detector

In [ ]:
!./darknet detector train data/obj.data cfg/yolov4-tiny-obj.cfg yolov4-tiny.conv.29 -map -dont_show 

Streaming output truncated to the last 5000 lines.
 (next mAP calculation at 3560 iterations) 
 Last accuracy mAP@0.50 = 82.47 %, best = 83.72 % 
 3479: 0.899479, 0.804174 avg loss, 0.002610 rate, 1.111748 seconds, 222656 images, 2.644768 hours left
Loaded: 0.435313 seconds - performance bottleneck on CPU or Disk HDD/SSD
v3 (iou loss, Normalizer: (iou: 0.07, obj: 1.00, cls: 1.00) Region 30 Avg (IOU: 0.782798), count: 144, class_loss = 1.179105, iou_loss = 0.529242, total_loss = 1.708346 
v3 (iou loss, Normalizer: (iou: 0.07, obj: 1.00, cls: 1.00) Region 37 Avg (IOU: 0.576777), count: 12, class_loss = 0.207862, iou_loss = 0.277332, total_loss = 0.485194 
 total_bbox = 672458, rewritten_bbox = 0.721235 % 

 (next mAP calculation at 3560 iterations) 
 Last accuracy mAP@0.50 = 82.47 %, best = 83.72 % 
 3480: 0.693834, 0.793140 avg loss, 0.002610 rate, 1.119004 seconds, 222720 images, 2.640102 hours left
Loaded: 0.398531 seconds - performance bottleneck on CPU or Disk HDD/SSD
v3 (iou loss, 

#### Test weights file

In [ ]:
%cp "/content/drive/MyDrive/MLData/tiny-yolov4-data/yolov4-tiny-obj_last_v1.weights" "/content/darknet/backup"

In [ ]:
!./darknet detector map data/obj.data cfg/yolov4-tiny-obj.cfg backup/yolo-obj_7000.weights